<a href="https://colab.research.google.com/github/shivani1912/eBay_ETL/blob/master/eBay_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective
Build an ETL pipeline to extract, transform and load eBay iPad 4 product listing into SQL database.
Analysis the difference between sponsored and non-sponsored product listings

In [ ]:
import string
import os
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup


<h6> Search for buy-it-now listings of ipad4 and limit the number of items to 100 per page. 

For the first 10 pages of 100 items/page, we save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as the code code. (One URL per line in each file)

In [ ]:
#querying for ipad4 and saving the search pages to file (Professor recommended always saving html pages to file before processing them)
base_url = "https://www.ebay.com/sch/i.html?_nkw=ipad4&LH_BIN=1&_ipg=100&_pgn="
headers = {'User-Agent': 'Mozilla/5.0'}
#Request to the server
a=list(range(1,11))
for number in a:
        user_agent='Mozilla/5.0'
        url=base_url+str(number)
        response= requests.get(url , headers={'User-Agent': user_agent})
        pg_text=response.text
        filename = 'ebay_ipad4_pg'+str(number)+'.htm'
        print(filename)
        with open(filename,'w') as file:
            file.write(str(response.content))
 

ebay_ipad4_pg1.htm
ebay_ipad4_pg2.htm
ebay_ipad4_pg3.htm
ebay_ipad4_pg4.htm
ebay_ipad4_pg5.htm
ebay_ipad4_pg6.htm
ebay_ipad4_pg7.htm
ebay_ipad4_pg8.htm
ebay_ipad4_pg9.htm
ebay_ipad4_pg10.htm


In [ ]:

#Copying all sponsored links to sponosred.txt file
a=list(range(1,11))
with open('sponsored.txt','w') as file:
    for number in a:
        filename = 'ebay_ipad4_pg'+str(number)+'.htm'
        with open(filename, 'r') as f:
            text = f.read()
            soup = BeautifulSoup(text, 'html.parser') 
            items = soup.find_all('div', attrs={'class':'s-item__info clearfix'})
            for item in items:
                a = item.find("a")
                if not a:
                    continue 
                text = item.find("span").get_text()
                if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):
                        file.write(a['href']+"\n")

In [ ]:
file.close()

In [ ]:

#Copying all non-sponsored links to non-sponosred.txt file
a=list(range(1,11))
with open('non-sponsored.txt','w') as file:
    for number in a:
        filename = 'ebay_ipad4_pg'+str(number)+'.htm'
        with open(filename, 'r') as f:
            text = f.read()
            soup = BeautifulSoup(text, 'html.parser') 
            items = soup.find_all('div', attrs={'class':'s-item__info clearfix'})
            for item in items:
                a = item.find("a")
                if not a:
                    continue 
                text = item.find("span").get_text()
                if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):
                        continue
                else:
                        file.write(a['href']+"\n")
  

In [ ]:
file.close()

<h6>  We create two folders in the same directory as the code and name them "sponsored" and "non-sponsored". We then write a program that opens the two files (sponsored.txt, non-sponsored.txt) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [ ]:
import os

# creating sponsored folder
try:
    os.mkdir('sponsored')
except OSError:
    print ("Creation of the directory failed")
else:
    print ("Successfully created the directory")

Successfully created the directory


In [ ]:
mypath=os.getcwd() 

In [ ]:

#Downloading all sponsored links to sponosred folder
a=list(range(1,11))
file=open('sponsored.txt','r')
lines = file.readlines()
os.chdir("sponsored")
for line in lines:
    url = line
    headers = {'User-Agent': 'Mozilla/5.0'}
    user_agent='Mozilla/5.0'
    time.sleep(2)
    response= requests.get(url , headers={'User-Agent': user_agent})
    pg_text=response.text
    soup = BeautifulSoup(pg_text, 'html.parser') 
    item = soup.find('div', attrs={'id':'descItemNumber'})
    item_id=item.get_text()
    filename = item_id+'.htm'
    with open(filename,'w',encoding="utf-8") as f:
            f.write(str(response.text))
            
  

In [ ]:
os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))

In [ ]:
import os
# creating non-sponsored folder
try:
    os.mkdir('non-sponsored')
except OSError:
    print ("Creation of the directory failed")
else:
    print ("Successfully created the directory")


Successfully created the directory


In [ ]:
#Downloading all non-sponsored links to non-sponsored folder
a=list(range(1,11))
file=open('non-sponsored.txt','r')
lines = file.readlines()
os.chdir("non-sponsored")
for line in lines:
    url = line
    headers = {'User-Agent': 'Mozilla/5.0'}
    user_agent='Mozilla/5.0'
    time.sleep(2)
    response= requests.get(url , headers={'User-Agent': user_agent})
    pg_text=response.text
    soup = BeautifulSoup(pg_text, 'html.parser') 
    item = soup.find('div', attrs={'id':'descItemNumber'})
    item_id=item.get_text()
    filename = item_id+'.htm'
    with open(filename,'w',encoding="utf-8") as f:
            f.write(response.text)
         

<h6>d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In your code, highlight the selector command you choose to obtain each element using comments.

In [ ]:
#Creating a dataframe to store the data
item_df=pd.DataFrame(columns=["item_id","sponsored","seller_name","seller_feedback_score","item_price","item_price_currency","items_sold","title","best_offer","returns","shipping_price_computed","shipping_price","shipping_price_currency","condition"])


In [ ]:
#non-sponsored files getting the data

for filename in os.listdir(os.getcwd()):
    with open(filename, 'r', encoding='utf-8') as f:
            text = f.read()
            
            soup = BeautifulSoup(text, 'html.parser') 
            #item_id selected by div-'id':'descItemNumber'
            item_id=soup.find('div', attrs={'id':'descItemNumber'}).get_text()
          
            
            #seller_name selected by span,'class':'mbg-nw'
            seller_name = soup.find('span', attrs={'class':'mbg-nw'}).get_text()
           
            
            #seller_feedback_score selected by span,'class':'mbg-l'
            seller_feedback_score = soup.find('span', attrs={'class':'mbg-l'}).get_text()
            seller_feedback_score=seller_feedback_score.replace("(","")
            seller_feedback_score=seller_feedback_score.strip(")")
            seller_feedback_score=seller_feedback_score.strip()
           
            
            #item_price selected by span, 'class':'notranslate' & div,'id':'vi-mskumap-none'
            item_price_text= soup.find('span', attrs={'class':'notranslate'}).get_text()
            if((re.search("[$|US].*",item_price_text))):
                temp = re.findall(r'\d+',item_price_text) 
                res = list(map(int, temp)) 
                item_price=float((res[0])*100+res[1])
                item_price_currency="USD"
                
            else:
                if(item_price_text.isspace()):
                    item_price_text= soup.find('div', attrs={'id':'vi-mskumap-none'}).find("span").get_text()
                temp5 = re.findall(r'\d+',item_price_text) 
                res5 = list(map(int, temp5)) 
                item_price=float((res5[0])*100+res5[1])
                item_price_currency=re.sub('[0-9.]+', " ",item_price_text)
                
             #items_sold selected by a,'class':'vi-txt-underline'
            items_sold_check=soup.find('a', attrs={'class':'vi-txt-underline'})
            if items_sold_check is None:
                items_sold=float("NAN")
            else:
                items_sold=soup.find('a', attrs={'class':'vi-txt-underline'}).get_text()        
        
            
            # title selected by h1,'id':'itemTitle'
            title= soup.find('h1', attrs={'id':'itemTitle'}).get_text()
            title=title.strip("Details about")
         
            
            #returns_alldata by span,'class':'w2b-sgl'
            returns_alldata= soup.find_all('span', attrs={'class':'w2b-sgl'})
            for line in returns_alldata:
                x=re.search(".*[R|r]eturns.*",line.get_text())
                if(x):
                    returns_text=line.get_text()
                    if(returns_text=="No returns"):
                        returns=0
                    elif((returns_text=="Returns accepted")|(returns_text=="30-day returns")):
                        returns=1
                    else:
                        continue
            
            #shipping_price by span,id':'shSummary'
            shipping_price_text= soup.find('span', attrs={'id':'shSummary'})
            if shipping_price_text is None:
                shipping_price_text=None
            else:
                shipping_price_text=shipping_price_text.find("span").get_text()
                if((re.search(".*[FREE|Free].*",shipping_price_text))):
                    shipping_price_computed=0
                    shipping_price=0
                    shipping_price_currency="USD"
                elif(shipping_price_text=="Varies based on location and shipping method"):
                    shipping_price_computed=1
                    shipping_price=None
                    shipping_price_currency="USD"
                elif((re.search("$.*",shipping_price_text))):
                    temp1 = re.findall(r'\d+',shipping_price_text) 
                    res1 = list(map(int, temp1)) 
                    shipping_price_computed=0
                    shipping_price=float((res1[0])*100+res1[1])
                    shipping_price_currency="USD"
                else:    
                    temp4 = re.findall(r'\d+',shipping_price_text) 
                    res4 = list(map(int, temp4)) 
                    shipping_price_computed=0
                    shipping_price=float((res4[0])*100+res4[1])
                    shipping_price_currency=re.sub('[0-9.]+', " ",shipping_price_text)
            
                
            #best_offer by div,'class':'vi-bbox-dspn u-flL lable boLable'
            best_offer_check=soup.find('div', attrs={'class':'vi-bbox-dspn u-flL lable boLable'})
            if best_offer_check is None:
                best_offer=float("NAN")
            else:
                best_offer=soup.find('div', attrs={'class':'vi-bbox-dspn u-flL lable boLable'}).get_text()    
           
            
            #condition by div,attrs={'itemprop':'itemCondition'}
            condition= soup.find('div', attrs={'itemprop':'itemCondition'}).get_text()
           
            sponsored="NON-SPONSORED"
            
            new_row=pd.DataFrame([[item_id,sponsored,seller_name,seller_feedback_score,item_price,item_price_currency,items_sold,title,best_offer,returns,shipping_price_computed,shipping_price,shipping_price_currency,condition]],
                                columns=["item_id","sponsored","seller_name","seller_feedback_score","item_price","item_price_currency","items_sold","title","best_offer","returns","shipping_price_computed","shipping_price","shipping_price_currency","condition"])

           
            item_df=item_df.append(new_row)
            
   

In [ ]:
f.close()

In [ ]:
os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))

In [ ]:

os.chdir("sponsored")

In [ ]:
#sponsored files getting the data

for filename in os.listdir(os.getcwd()):
    with open(filename, 'r', encoding='utf-8') as f:
            text = f.read()
            
            soup = BeautifulSoup(text, 'html.parser') 
            #item_id selected by div-'id':'descItemNumber'
            item_id=soup.find('div', attrs={'id':'descItemNumber'}).get_text()
          
            
            #seller_name selected by span,'class':'mbg-nw'
            seller_name = soup.find('span', attrs={'class':'mbg-nw'}).get_text()
           
            
            #seller_feedback_score selected by span,'class':'mbg-l'
            seller_feedback_score = soup.find('span', attrs={'class':'mbg-l'}).get_text()
            seller_feedback_score=seller_feedback_score.replace("(","")
            seller_feedback_score=seller_feedback_score.strip(")")
            seller_feedback_score=seller_feedback_score.strip()
           
            
            #item_price selected by span, 'class':'notranslate' & div,'id':'vi-mskumap-none'
            item_price_text= soup.find('span', attrs={'class':'notranslate'}).get_text()
            if((re.search("[$|US].*",item_price_text))):
                temp = re.findall(r'\d+',item_price_text) 
                res = list(map(int, temp)) 
                item_price=float((res[0])*100+res[1])
                item_price_currency="USD"
                
            else:
                if(item_price_text.isspace()):
                    item_price_text= soup.find('div', attrs={'id':'vi-mskumap-none'}).find("span").get_text()
                temp5 = re.findall(r'\d+',item_price_text) 
                res5 = list(map(int, temp5)) 
                item_price=float((res5[0])*100+res5[1])
                item_price_currency=re.sub('[0-9.]+', " ",item_price_text)
                
             #items_sold selected by a,'class':'vi-txt-underline'
            items_sold_check=soup.find('a', attrs={'class':'vi-txt-underline'})
            if items_sold_check is None:
                items_sold=float("NAN")
            else:
                items_sold=soup.find('a', attrs={'class':'vi-txt-underline'}).get_text()        
        
            
            # title selected by h1,'id':'itemTitle'
            title= soup.find('h1', attrs={'id':'itemTitle'}).get_text()
            title=title.strip("Details about")
         
            
            #returns_alldata by span,'class':'w2b-sgl'
            returns_alldata= soup.find_all('span', attrs={'class':'w2b-sgl'})
            for line in returns_alldata:
                x=re.search(".*[R|r]eturns.*",line.get_text())
                if(x):
                    returns_text=line.get_text()
                    if(returns_text=="No returns"):
                        returns=0
                    elif((returns_text=="Returns accepted")|(returns_text=="30-day returns")):
                        returns=1
                    else:
                        continue
            
            #shipping_price by span,id':'shSummary'
            shipping_price_text= soup.find('span', attrs={'id':'shSummary'})
            if shipping_price_text is None:
                shipping_price_text=None
            else:
                shipping_price_text=shipping_price_text.find("span").get_text()
                if((re.search(".*[FREE|Free].*",shipping_price_text))):
                    shipping_price_computed=0
                    shipping_price=0
                    shipping_price_currency="USD"
                elif(shipping_price_text=="Varies based on location and shipping method"):
                    shipping_price_computed=1
                    shipping_price=None
                    shipping_price_currency="USD"
                elif((re.search("$.*",shipping_price_text))):
                    temp1 = re.findall(r'\d+',shipping_price_text) 
                    res1 = list(map(int, temp1)) 
                    shipping_price_computed=0
                    shipping_price=float((res1[0])*100+res1[1])
                    shipping_price_currency="USD"
                else:    
                    temp4 = re.findall(r'\d+',shipping_price_text) 
                    res4 = list(map(int, temp4)) 
                    shipping_price_computed=0
                    shipping_price=float((res4[0])*100+res4[1])
                    shipping_price_currency=re.sub('[0-9.]+', " ",shipping_price_text)
            
                
            #best_offer by div,'class':'vi-bbox-dspn u-flL lable boLable'
            best_offer_check=soup.find('div', attrs={'class':'vi-bbox-dspn u-flL lable boLable'})
            if best_offer_check is None:
                best_offer=float("NAN")
            else:
                best_offer=soup.find('div', attrs={'class':'vi-bbox-dspn u-flL lable boLable'}).get_text()    
           
            
            #condition by div,attrs={'itemprop':'itemCondition'}
            condition= soup.find('div', attrs={'itemprop':'itemCondition'}).get_text()
           
            sponsored="SPONSORED"
            
            new_row=pd.DataFrame([[item_id,sponsored,seller_name,seller_feedback_score,item_price,item_price_currency,items_sold,title,best_offer,returns,shipping_price_computed,shipping_price,shipping_price_currency,condition]],
                                columns=["item_id","sponsored","seller_name","seller_feedback_score","item_price","item_price_currency","items_sold","title","best_offer","returns","shipping_price_computed","shipping_price","shipping_price_currency","condition"])

           
            item_df=item_df.append(new_row)
            
   

In [ ]:
#data wrangling

#items_sold getting the number
item_df['items_sold']=item_df['items_sold'].str.replace(r'\D', '')

#items_sold making a copy sqldata
sqldata=item_df

#replacing nan with None
sqldata = sqldata.replace({pd.np.nan: None})

#converting numeric
sqldata['items_sold']=pd.to_numeric(sqldata['items_sold'])
sqldata['seller_feedback_score']=pd.to_numeric(sqldata['seller_feedback_score'])
sqldata['item_price']=pd.to_numeric(sqldata['item_price'])
sqldata['shipping_price']=pd.to_numeric(sqldata['shipping_price'])

#data wrangling converting best_offer from text to 1 indicating availability of best offer
sqldata['best_offer']=sqldata['best_offer'].fillna(0)
sqldata.loc[(sqldata['best_offer'] == 'Best Offer:'), 'best_offer'] = 1 

#Getting all prices' currencies the US $ to USD
sqldata.loc[(sqldata['item_price_currency'] == 'US $ '), 'item_price_currency'] = 'USD'
sqldata.loc[(sqldata['shipping_price_currency'] == 'US $ '), 'item_price_currency'] = 'USD'

In [ ]:
#no of unique items
sqldata['item_id'].nunique()

748

<h6> Connect to SQL (here it is MySQL). Create a database and name it "eBay". Save the information of product listings' details into a single table named "eBay_items" . This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  

Note: Already converted the Dollars into cents before inserting into the table.None in the dataframe are converted to NULL by mySQL

In [ ]:
#connecting to workbench mysql
import mysql.connector
from mysql.connector import errorcode

try:
  cnx = mysql.connector.connect(user='root',
                                password='Swati9000', host='localhost')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
   if (cnx.is_connected()):
        print("Connected")
        db_Info = cnx.get_server_info()
        print("Connected to MySQL database... MySQL Server version on ",db_Info)
  

Connected
Connected to MySQL database... MySQL Server version on  8.0.17


 creates a database named eBay

In [ ]:
# creating database_cursor to perform SQL operation
db_cursor = cnx.cursor()
# executing cursor with execute method and pass SQL query
db_cursor.execute("CREATE DATABASE eBay")
# get list of all databases
db_cursor.execute("SHOW DATABASES")
#print all databases
for db in db_cursor:
	print(db)

('accountspayable',)
('avidreader',)
('bowlingleagueexample',)
('br',)
('colonial',)
('ebay',)
('entertainmentagencyexample',)
('information_schema',)
('my_first_db',)
('mysql',)
('oact',)
('peoplefun',)
('performance_schema',)
('recipesdatabase',)
('sakila',)
('salesordersdatabase',)
('sys',)
('ucdavis',)
('world',)


In [ ]:
db_cursor.close()
cnx.close()

creates a table eBay_items

In [ ]:
#Here creating database table as eBay_items
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay"
  )
db_cursor = db_connection.cursor()
db_cursor.execute("CREATE TABLE eBay_items(record_id INT NOT NULL AUTO_INCREMENT,item_id VARCHAR(255),title VARCHAR(600),sponsored VARCHAR(20),seller_name VARCHAR(255),seller_feedback_score INT(10),item_price INT(10),item_price_currency VARCHAR(4),items_sold INT(5),best_offer_available VARCHAR(255),returns_allowed BOOLEAN,shipping_price_computed BOOLEAN, shipping_price INT(5), shipping_price_currency VARCHAR(4),item_condition VARCHAR(255),PRIMARY KEY(record_id))")
   
#Get database table'
db_cursor.execute("SHOW TABLES")
for table in db_cursor:
	print(table)
db_cursor.close()
db_connection.close()

('ebay_items',)


In [ ]:
#replacing nan with None
sqldata = sqldata.astype(object).where(pd.notnull(sqldata), None)

In [ ]:
#getting each row in the datframe item_df and inserting into the sql table omdb
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay"
  )
db_cursor = db_connection.cursor()
for index, row in sqldata.iterrows():
    #inserting into the table 
    try:
        sql_query = "INSERT INTO eBay_items(item_id,title,sponsored,seller_name,seller_feedback_score,item_price,item_price_currency,items_sold,best_offer_available,returns_allowed,shipping_price_computed, shipping_price , shipping_price_currency,item_condition) VALUES(%s, %s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s,%s,%s)"
        record=(row['item_id'],row['title'],row['sponsored'],row['seller_name'],row['seller_feedback_score'], row['item_price'],row['item_price_currency'],row['items_sold'],row['best_offer'],row['returns'],row['shipping_price_computed'],row['shipping_price'],row['shipping_price_currency'],row['condition'])
        db_cursor.execute(sql_query,record)
    except mysql.connector.IntegrityError as err:
        print("Error: {}".format(err))

In [ ]:
db_connection.commit()

In [ ]:
#validating whether records were inserted
db_cursor = db_connection.cursor()
db_cursor.execute("select * from eBay_items")
records = db_cursor.fetchall()
print("Total number of rows in eBay_items is: ", db_cursor.rowcount)
db_cursor.close()
db_connection.close()

Total number of rows in eBay_items is:  748


<h6>f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

Note: mySQL ignores NULL which calculating statistics
Source: https://www.mysqltutorial.org/mysql-avg/

In [ ]:
#Statistics on seller_feedback score
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay"
  )
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition, max(seller_feedback_score) as 'Max_Seller_Score', min(seller_feedback_score) as 'Min_Seller_Score', round(avg(seller_feedback_score),2) as 'Mean_Seller_Score',round(stddev(seller_feedback_score),2) as 'Standard_Deviation_Seller_Score' from ebay_items group by sponsored,item_condition")
records = db_cursor.fetchall()
seller_feedback=pd.DataFrame(records, columns=["sponsored","item_condition","Max","Min","Mean","Std_Deviations"])
print(seller_feedback)
db_cursor.close()

        sponsored            item_condition      Max   Min      Mean  \
0   NON-SPONSORED                       New   102022    59   6556.14   
1   NON-SPONSORED        Seller refurbished  2266102     1  95519.98   
2   NON-SPONSORED                      Used   311772     0  13133.40   
3   NON-SPONSORED  Manufacturer refurbished   382304     0  69028.93   
4   NON-SPONSORED  For parts or not working    99895     0  16904.96   
5   NON-SPONSORED                  Open box   106215     3  39286.10   
6       SPONSORED                       New    23265  1248   9660.00   
7       SPONSORED                      Used    18616     0   8003.44   
8       SPONSORED  For parts or not working   102021    32  36383.33   
9       SPONSORED  Manufacturer refurbished    84743    59  55094.07   
10      SPONSORED                  Open box   102021   530  24054.00   
11      SPONSORED        Seller refurbished    84743   673  24073.69   

    Std_Deviations  
0         20998.26  
1        323533.71  


True

In [ ]:
#Statistics on item_price
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition,item_price_currency, max(item_price), min(item_price), round(avg(item_price),2) ,round(stddev(item_price),2)  from ebay_items group by sponsored,item_condition,item_price_currency")
records = db_cursor.fetchall()
item_price=pd.DataFrame(records, columns=["sponsored","item_condition","item_price_currency","Max","Min","Mean","Std_Deviations"])
print(item_price)
db_cursor.close()

        sponsored            item_condition item_price_currency    Max    Min  \
0   NON-SPONSORED                       New                 USD  99998   3599   
1   NON-SPONSORED        Seller refurbished                 USD  99999   7995   
2   NON-SPONSORED                      Used                 USD  99999    100   
3   NON-SPONSORED  Manufacturer refurbished                 USD  82900   1899   
4   NON-SPONSORED  For parts or not working                 USD  22222    999   
5   NON-SPONSORED                       New                GBP     399    399   
6   NON-SPONSORED                  Open box                 USD  75000   8995   
7   NON-SPONSORED                      Used                GBP   19900  19900   
8       SPONSORED                       New                GBP     399    285   
9       SPONSORED                      Used                 USD  21799   4999   
10      SPONSORED  For parts or not working                 USD  12000   1500   
11      SPONSORED  Manufactu

True

In [ ]:
#Statistics on shipping_price
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition,shipping_price_currency, max(shipping_price), min(shipping_price), round(avg(shipping_price),2) ,round(stddev(shipping_price),2)  from ebay_items group by sponsored,item_condition,shipping_price_currency")
records = db_cursor.fetchall()
shipping_price=pd.DataFrame(records, columns=["sponsored","item_condition","shipping_price_currency","Max","Min","Mean","Std_Deviations"])
print(shipping_price)
db_cursor.close()

        sponsored            item_condition shipping_price_currency   Max  \
0   NON-SPONSORED                       New                     USD  3850   
1   NON-SPONSORED        Seller refurbished                     USD  1995   
2   NON-SPONSORED                      Used                     USD  4995   
3   NON-SPONSORED  Manufacturer refurbished                     USD     0   
4   NON-SPONSORED  For parts or not working                     USD  2425   
5   NON-SPONSORED                  Open box                     USD  1400   
6       SPONSORED                       New                     USD   300   
7       SPONSORED                      Used                     USD   495   
8       SPONSORED  For parts or not working                     USD  2000   
9       SPONSORED  Manufacturer refurbished                     USD     0   
10      SPONSORED                  Open box                     USD     0   
11      SPONSORED        Seller refurbished                     USD  1505   

True

In [ ]:
#items_sold
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition, max(items_sold), min(items_sold), round(avg(items_sold),2) ,round(stddev(items_sold),2)  from ebay_items group by sponsored,item_condition")
records = db_cursor.fetchall()
items_sold=pd.DataFrame(records, columns=["sponsored","item_condition","Max","Min","Mean","Std_Deviations"])
print(items_sold)
db_cursor.close()

        sponsored            item_condition    Max  Min    Mean  \
0   NON-SPONSORED                       New     29    1    8.67   
1   NON-SPONSORED        Seller refurbished   1256    1  130.78   
2   NON-SPONSORED                      Used    536    1   45.00   
3   NON-SPONSORED  Manufacturer refurbished  18682    1  549.80   
4   NON-SPONSORED  For parts or not working      3    1    2.00   
5   NON-SPONSORED                  Open box    753    1  158.42   
6       SPONSORED                       New    140   85  112.50   
7       SPONSORED                      Used    134    2   51.80   
8       SPONSORED  For parts or not working      7    1    4.00   
9       SPONSORED  Manufacturer refurbished   3294    2  412.76   
10      SPONSORED                  Open box    147    6   73.67   
11      SPONSORED        Seller refurbished   2379    4  306.04   

    Std_Deviations  
0             8.72  
1           262.20  
2           109.30  
3          2612.34  
4             1.00  
5 

True

In [ ]:
#returns allowed
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition, max(returns_allowed), min(returns_allowed), round(avg(returns_allowed),2) ,round(stddev(returns_allowed),2)  from ebay_items group by sponsored,item_condition")
records = db_cursor.fetchall()
returns_allowed=pd.DataFrame(records, columns=["sponsored","item_condition","Max","Min","Mean","Std_Deviations"])
print(returns_allowed)
db_cursor.close()

        sponsored            item_condition  Max  Min  Mean  Std_Deviations
0   NON-SPONSORED                       New    1    0  0.95            0.21
1   NON-SPONSORED        Seller refurbished    1    0  0.95            0.22
2   NON-SPONSORED                      Used    1    0  0.50            0.50
3   NON-SPONSORED  Manufacturer refurbished    1    0  0.93            0.26
4   NON-SPONSORED  For parts or not working    1    0  0.43            0.49
5   NON-SPONSORED                  Open box    1    0  0.79            0.41
6       SPONSORED                       New    1    0  0.75            0.43
7       SPONSORED                      Used    1    0  0.78            0.42
8       SPONSORED  For parts or not working    1    0  0.67            0.47
9       SPONSORED  Manufacturer refurbished    1    1  1.00            0.00
10      SPONSORED                  Open box    1    1  1.00            0.00
11      SPONSORED        Seller refurbished    1    1  1.00            0.00


True

In [ ]:
#best offer
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition, max(best_offer_available), min(best_offer_available), round(avg(best_offer_available),2) ,round(stddev(best_offer_available),2)  from ebay_items group by sponsored,item_condition")

records = db_cursor.fetchall()
best_offer=pd.DataFrame(records, columns=["sponsored","item_condition","Max","Min","Mean","Std_Deviations"])
print(best_offer)

        sponsored            item_condition Max Min  Mean  Std_Deviations
0   NON-SPONSORED                       New   1   0  0.27            0.45
1   NON-SPONSORED        Seller refurbished   1   0  0.08            0.27
2   NON-SPONSORED                      Used   1   0  0.48            0.50
3   NON-SPONSORED  Manufacturer refurbished   1   0  0.10            0.30
4   NON-SPONSORED  For parts or not working   1   0  0.32            0.47
5   NON-SPONSORED                  Open box   1   0  0.17            0.37
6       SPONSORED                       New   1   0  0.25            0.43
7       SPONSORED                      Used   1   0  0.44            0.50
8       SPONSORED  For parts or not working   1   0  0.42            0.49
9       SPONSORED  Manufacturer refurbished   1   0  0.04            0.19
10      SPONSORED                  Open box   0   0  0.00            0.00
11      SPONSORED        Seller refurbished   1   0  0.08            0.27


In [ ]:
#sellername, item_titles
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition,count(distinct(seller_name)), count(distinct(title)) from ebay_items group by sponsored,item_condition")

records = db_cursor.fetchall()
cat_var=pd.DataFrame(records, columns=["sponsored","item_condition","sellers","unique_items"])
print(cat_var)

        sponsored            item_condition  sellers  unique_items
0   NON-SPONSORED  For parts or not working       57            86
1   NON-SPONSORED  Manufacturer refurbished       29            78
2   NON-SPONSORED                       New       11            22
3   NON-SPONSORED                  Open box       20            42
4   NON-SPONSORED        Seller refurbished       40            92
5   NON-SPONSORED                      Used      241           269
6       SPONSORED  For parts or not working       10            12
7       SPONSORED  Manufacturer refurbished        8            27
8       SPONSORED                       New        4             4
9       SPONSORED                  Open box        4             5
10      SPONSORED        Seller refurbished       10            26
11      SPONSORED                      Used        8             9


In [ ]:
#sponsored_best offer
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored, max(best_offer_available), min(best_offer_available), round(avg(best_offer_available),2) ,round(stddev(best_offer_available),2)  from ebay_items group by sponsored")

records = db_cursor.fetchall()
sponsored_best_offer=pd.DataFrame(records, columns=["sponsored","Max","Min","Mean","Std_Deviations"])
print(sponsored_best_offer)

       sponsored Max Min  Mean  Std_Deviations
0  NON-SPONSORED   1   0  0.32            0.47
1      SPONSORED   1   0  0.16            0.36


In [ ]:
#sponsored_ items_sold
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored, max(items_sold), min(items_sold), round(avg(items_sold),2) ,round(stddev(items_sold),2)  from ebay_items group by sponsored")

records = db_cursor.fetchall()
sponsored_items_sold=pd.DataFrame(records, columns=["sponsored","Max","Min","Mean","Std_Deviations"])
print(sponsored_items_sold)

       sponsored    Max  Min    Mean  Std_Deviations
0  NON-SPONSORED  18682    1  219.98         1392.46
1      SPONSORED   3294    1  301.18          556.69


In [ ]:
#sponsored_ item_price
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored, max(item_price), min(item_price), round(avg(item_price),2) ,round(stddev(item_price),2)  from ebay_items group by sponsored")

records = db_cursor.fetchall()
sponsored_item_price=pd.DataFrame(records, columns=["sponsored","Max","Min","Mean","Std_Deviations"])
print(sponsored_item_price)

       sponsored    Max  Min      Mean  Std_Deviations
0  NON-SPONSORED  99999  100  17287.81        17402.78
1      SPONSORED  39000  285  12346.71         5920.43


In [ ]:
#sponsored_ shipping_price
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored, max(shipping_price), min(shipping_price), round(avg(shipping_price),2) ,round(stddev(shipping_price),2)  from ebay_items group by sponsored")

records = db_cursor.fetchall()
sponsored_shipping_price=pd.DataFrame(records, columns=["sponsored","Max","Min","Mean","Std_Deviations"])
print(sponsored_shipping_price)

       sponsored   Max  Min    Mean  Std_Deviations
0  NON-SPONSORED  4995    0  181.15          530.85
1      SPONSORED  2000    0   72.29          330.32


In [ ]:
#sponsored_ returns
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored, max(returns_allowed), min(returns_allowed), round(avg(returns_allowed),2) ,round(stddev(returns_allowed),2)  from ebay_items group by sponsored")

records = db_cursor.fetchall()
sponsored_returns_allowed=pd.DataFrame(records, columns=["sponsored","Max","Min","Mean","Std_Deviations"])
print(sponsored_returns_allowed)

       sponsored  Max  Min  Mean  Std_Deviations
0  NON-SPONSORED    1    0  0.64            0.48
1      SPONSORED    1    0  0.92            0.28


In [ ]:
#sponsored_ item_condition
db_connection = mysql.connector.connect(
 user='root',password='Swati9000', host='localhost',database="eBay")
db_cursor = db_connection.cursor()
db_cursor.execute("select sponsored,item_condition, count(item_id) from ebay_items group by sponsored, item_condition")

records = db_cursor.fetchall()
sponsored_item_condition=pd.DataFrame(records, columns=["sponsored","item_condition","No_of_Items"])
print(sponsored_item_condition)

        sponsored            item_condition  No_of_Items
0   NON-SPONSORED                       New           22
1   NON-SPONSORED        Seller refurbished           99
2   NON-SPONSORED                      Used          331
3   NON-SPONSORED  Manufacturer refurbished           80
4   NON-SPONSORED  For parts or not working           91
5   NON-SPONSORED                  Open box           42
6       SPONSORED                       New            4
7       SPONSORED                      Used            9
8       SPONSORED  For parts or not working           12
9       SPONSORED  Manufacturer refurbished           27
10      SPONSORED                  Open box            5
11      SPONSORED        Seller refurbished           26


In [ ]:
db_connection.close()

<h6> Conclusions: Difference between Sponsored & Non-sponsored

Number of sellers,Number of best offers available, Shipping price, Item price all seems to be higher for Non sponsored as compared to sponsored , but this could be due to the fact there are more number of nonsponsored items as compared to sponsored items.

Returns allowed is higher for sponsored as compared to non sponsored (More mean and less deviation)

Number of used items seem to be much higher for Non sponsored as compared to Sponsored.

Returns allowed and Item conditions are 2 variables which can indicate whether an item is Sponsored or Not Sponsored